## **Capítulo 6: Análise de Carteiras**

Até agora, analisamos ativos individualmente. No entanto, investidores raramente colocam todo o seu dinheiro em um único ativo; eles constroem uma **carteira (ou portfólio)**.

Neste capítulo, vamos mergulhar nos fundamentos da **Teoria Moderna do Portfólio**, cujo princípio central é a **diversificação**. A ideia é que, ao combinar diferentes ativos, podemos construir uma carteira que tem um risco menor do que a simples soma dos riscos individuais dos seus componentes.

### **Retorno e Risco de uma Carteira com 2 Ativos**

Vamos começar com um caso simples: uma carteira com apenas dois ativos.

#### **Retorno da Carteira**
O retorno esperado de uma carteira é simplesmente a soma ponderada dos retornos esperados de cada ativo. A fórmula é:

$r_c = w_1r_1 + w_2r_2$

Onde:
* $r_c$ é o retorno da carteira.
* $r_1$ e $r_2$ são os retornos dos ativos 1 e 2.
* $w_1$ e $w_2$ são os pesos (percentuais) de cada ativo na carteira, e a soma deles deve ser 1 (ou 100%).

#### **Risco da Carteira**
O risco (volatilidade) da carteira é mais complexo. Ele não depende apenas do risco individual de cada ativo, mas também, e de forma crucial, da **correlação ($\rho$)** entre eles[cite: 738, 740]. É aqui que a mágica da diversificação acontece. A fórmula da variância da carteira é:

$\sigma_{c}^{2} = w_{1}^{2}\sigma_{1}^{2} + w_{2}^{2}\sigma_{2}^{2} + 2w_{1}w_{2}\sigma_{1}\sigma_{2}\rho_{12}$

Onde:
* $\sigma_c^2$ é a variância da carteira (o risco ao quadrado).
* $\sigma_1^2$ e $\sigma_2^2$ são as variâncias dos ativos 1 e 2.
* $\rho_{12}$ é a correlação entre os retornos dos dois ativos.

### **Exemplo Prático: Construindo a Fronteira Eficiente**

Vamos agora aplicar essa teoria para construir a **Fronteira Eficiente**. Ela é um gráfico que mostra todas as combinações possíveis de risco e retorno para uma carteira com os dois ativos, variando os pesos de 0% a 100%.

Usaremos dois ETFs como exemplo, assim como na apostila:
* `BOVA11.SA`: Replica o índice Ibovespa.
* `SPXI11.SA`: Replica o índice S&P 500 (bolsa americana) em Reais.

O primeiro passo é obter os dados e calcular os "ingredientes" da nossa fórmula: o retorno esperado, a volatilidade de cada ativo e a correlação entre eles.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Passo 1: Obter os Dados ---
tickers = ['BOVA11.SA', 'SPXI11.SA']
start_date = '2016-01-01'
end_date = '2019-12-31'

# 1. Baixamos o DataFrame completo
print("Baixando dados...")
dados_completos = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)

# 2. Selecionamos a coluna 'Close'
print("Selecionando a coluna 'Close'...")
precos = dados_completos['Close']

# 3. Removemos as linhas com dados faltantes
precos = precos.dropna()


# --- Passo 2: Calcular os Retornos Logarítmicos Diários ---
retornos = np.log(precos).diff().dropna()


# --- Passo 3: Calcular os "Ingredientes" Anualizados ---
retorno_esp_anual = retornos.mean() * 252
vol_anual = retornos.std() * np.sqrt(252)
matriz_corr = retornos.corr()

print("\n--- Retorno Esperado Anualizado (baseado no 'Close') ---")
print(retorno_esp_anual.to_string(float_format="{:.2%}".format))
print("\n--- Volatilidade Anualizada (Risco) ---")
print(vol_anual.to_string(float_format="{:.2%}".format))
print("\n--- Matriz de Correlação ---")
print(matriz_corr)

### **Desenhando a Fronteira Eficiente**

Agora, vamos aplicar as fórmulas de retorno e risco da carteira que vimos. A ideia é simular centenas de carteiras diferentes, variando os pesos entre os nossos dois ativos (`BOVA11.SA` e `SPXI11.SA`).

Por exemplo, uma carteira com 100% em BOVA11, outra com 99% em BOVA11 e 1% em SPXI11, e assim por diante, até 100% em SPXI11.

Ao calcular o risco e o retorno para cada uma dessas combinações e plotá-los em um gráfico, a curva que se forma é a **Fronteira Eficiente**. Ela mostra o máximo retorno que você pode obter para um dado nível de risco.

In [ ]:
# --- Passo 4: Simular as Carteiras e Desenhar a Fronteira ---

# Primeiro, vamos extrair os ingredientes que calculamos antes para variáveis simples
retorno_bova = retorno_esp_anual['BOVA11.SA']
retorno_spxi = retorno_esp_anual['SPXI11.SA']

vol_bova = vol_anual['BOVA11.SA']
vol_spxi = vol_anual['SPXI11.SA']

# A correlação é o valor na célula (0, 1) da matriz de correlação
corr = matriz_corr.iloc[0, 1]

# Criamos uma lista para guardar os resultados de cada carteira simulada
resultados_carteiras = []

# Geramos 100 pesos diferentes para o BOVA11, de 0 a 1 (0% a 100%)
pesos_bova = np.linspace(0, 1, 100)

# Loop para simular cada carteira
for peso_bova in pesos_bova:
    peso_spxi = 1 - peso_bova
    
    # Calcula o retorno da carteira
    retorno_carteira = peso_bova * retorno_bova + peso_spxi * retorno_spxi
    
    # Calcula a variância da carteira
    var_carteira = (peso_bova**2 * vol_bova**2) + \
                   (peso_spxi**2 * vol_spxi**2) + \
                   (2 * peso_bova * peso_spxi * vol_bova * vol_spxi * corr)
    
    # Calcula a volatilidade (risco) da carteira
    vol_carteira = np.sqrt(var_carteira)
    
    # Guarda os resultados
    resultados_carteiras.append({'retorno': retorno_carteira, 'vol': vol_carteira, 'peso_bova': peso_bova})

# Converte a lista de resultados em um DataFrame
fronteira_df = pd.DataFrame(resultados_carteiras)


# --- Encontrando a Carteira de Mínima Variância ---
# Localiza a carteira com a menor volatilidade (risco)
carteira_min_vol = fronteira_df.loc[fronteira_df['vol'].idxmin()]


# --- Plotando o Gráfico ---
plt.figure(figsize=(12, 8))

# Plota a fronteira eficiente
plt.plot(fronteira_df['vol'], fronteira_df['retorno'], 'b--', label='Fronteira Eficiente')

# Plota os ativos individuais
plt.scatter(vol_bova, retorno_bova, marker='o', color='red', s=100, label='BOVA11.SA')
plt.scatter(vol_spxi, retorno_spxi, marker='o', color='green', s=100, label='SPXI11.SA')

# Plota a carteira de mínima variância
plt.scatter(carteira_min_vol['vol'], carteira_min_vol['retorno'], marker='*', color='gold', s=200, label='Carteira de Mínima Variância')

plt.title('Fronteira Eficiente (BOVA11 vs SPXI11)', fontsize=16)
plt.xlabel('Volatilidade Anualizada (Risco)', fontsize=12)
plt.ylabel('Retorno Anualizado', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

print("--- Carteira de Mínima Variância ---")
print(f"Peso BOVA11: {carteira_min_vol['peso_bova']:.2%}")
print(f"Peso SPXI11: {1 - carteira_min_vol['peso_bova']:.2%}")
print(f"Retorno Esperado: {carteira_min_vol['retorno']:.2%}")
print(f"Risco (Volatilidade): {carteira_min_vol['vol']:.2%}")

## **6.3 Simulando a Performance da Carteira**

Com os pesos ótimos em mãos (no nosso caso, aproximadamente 40% para BOVA11 e 60% para SPXI11), podemos criar uma nova série de retornos que representa a nossa carteira.

O retorno diário da nossa carteira será a soma dos retornos diários de cada ativo, ponderados pelos seus respectivos pesos. Depois, podemos calcular o retorno acumulado dessa nova série para visualizar a performance ao longo do tempo.

In [ ]:
# --- Passo 5: Simular a Performance Histórica da Carteira de Mínima Variância ---

# Pegando os pesos que encontramos no passo anterior
peso_bova_otimo = carteira_min_vol['peso_bova']
peso_spxi_otimo = 1 - peso_bova_otimo

# 'retornos' é o nosso DataFrame com os retornos logarítmicos diários de BOVA11 e SPXI11
# Calculamos o retorno diário da nossa carteira ponderada
retornos['carteira_mvp'] = (retornos['BOVA11.SA'] * peso_bova_otimo) + \
                          (retornos['SPXI11.SA'] * peso_spxi_otimo)


# --- Calculando o Retorno Acumulado (Índice de Riqueza) da Carteira ---
# Usamos a soma acumulada dos log returns e a exponencial
retorno_acumulado_carteira = np.exp(retornos['carteira_mvp'].cumsum())


# --- Plotando o Gráfico de Performance ---
plt.figure(figsize=(12, 8))
retorno_acumulado_carteira.plot(label='Carteira de Mínima Variância')
plt.title('Performance Histórica da Carteira de Mínima Variância (2016-2019)', fontsize=16)
plt.ylabel('Índice de Riqueza (R$1,00 investido)')
plt.xlabel('Data')
plt.legend()
plt.grid(True)
plt.show()

## **6.4 A Importância do Rebalanceamento**

Um ponto importante a ser notado é que, ao longo do tempo, se um ativo da sua carteira se valoriza mais que o outro, os pesos originais (40%/60%) vão se desbalancear. Por exemplo, sua carteira pode acabar com 50%/50% naturalmente.

**Rebalanceamento** é o processo de comprar e vender ativos periodicamente (ex: a cada mês ou trimestre) para forçar a carteira a voltar aos seus pesos-alvo originais.

**Nota:** A simulação que fizemos acima, ao aplicar os mesmos pesos todos os dias, é uma aproximação que implicitamente assume um **rebalanceamento diário**. Na vida real, os custos de transação tornariam isso impraticável, e o rebalanceamento é feito com menos frequência.

# Fim da Apostila de Finanças com Python

Parabéns! Você chegou ao final da adaptação da apostila. Ao longo destes 6 capítulos, você construiu uma base sólida em finanças quantitativas usando Python.

Você aprendeu a:
* **Manipular dados** com as ferramentas essenciais do **Pandas** e **NumPy**.
* **Obter dados financeiros** de diversas fontes online com `yfinance` e `python-bcb`.
* **Calcular e analisar retornos** (simples e logarítmicos).
* **Medir e visualizar o risco** de ativos (Volatilidade, VaR, Drawdown).
* **Comparar múltiplos ativos** e suas características de risco-retorno.
* **Construir e analisar uma carteira otimizada** baseada na Teoria Moderna do Portfólio.

Este é um excelente ponto de partida. O mundo de finanças com Python é vasto, e agora você tem o conhecimento fundamental para explorar tópicos mais avançados, como backtesting de estratégias, machine learning para previsão de preços e muito mais.

**Ótimo trabalho!**